In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import spacy
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from alibi.explainers import AnchorTabular, AnchorText
from alibi.datasets import fetch_adult, fetch_movie_sentiment
from alibi.utils import gen_category_map, DistilbertBaseUncased, BertBaseUncased, RobertaBase

c:\Users\admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\admin\anaconda3\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Users\admin\anaconda3\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [2]:
adult = fetch_adult()
adult.keys()

dict_keys(['data', 'target', 'feature_names', 'target_names', 'category_map'])

In [3]:
adult['category_map']

{1: ['?',
  'Federal-gov',
  'Local-gov',
  'Never-worked',
  'Private',
  'Self-emp-inc',
  'Self-emp-not-inc',
  'State-gov',
  'Without-pay'],
 2: ['Associates',
  'Bachelors',
  'Doctorate',
  'Dropout',
  'High School grad',
  'Masters',
  'Prof-School'],
 3: ['Married', 'Never-Married', 'Separated', 'Widowed'],
 4: ['?',
  'Admin',
  'Blue-Collar',
  'Military',
  'Other',
  'Professional',
  'Sales',
  'Service',
  'White-Collar'],
 5: ['Husband',
  'Not-in-family',
  'Other-relative',
  'Own-child',
  'Unmarried',
  'Wife'],
 6: ['Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'],
 7: ['Female', 'Male'],
 11: ['?',
  'British-Commonwealth',
  'China',
  'Euro_1',
  'Euro_2',
  'Latin-America',
  'Other',
  'SE-Asia',
  'South-America',
  'United-States',
  'Yugoslavia']}

In [4]:
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

In [5]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:, :-1]
target = data_perm[:, -1]
idx = 30000
X_train, y_train = data[:idx, :], target[:idx]
X_test, y_test = data[idx:, :], target[idx:]